In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.optimize import root

def rotate_elastic_tensor(C, euler_angles):
    """
    Rotate 4th-order elasticity tensor using Euler angles (ZYZ convention).
    """
    R = Rotation.from_euler('zyz', euler_angles, degrees=True).as_matrix()
    C_rot = np.einsum('ia,jb,kc,ld,abcd->ijkl', R, R, R, R, C)
    return C_rot

# Elastic constants for Nickel (C11, C12, C44 in Pa)
C11, C12, C44 = 248.1e9, 154.9e9, 124.2e9
rho = 8900  # kg/m³

# Construct cubic elasticity tensor
C = np.zeros((3,3,3,3))
for i, j, k, l in np.ndindex(3,3,3,3):
    if i == j == k == l:
        C[i,j,k,l] = C11
    elif (i == k and j == l) or (i == l and j == k):
        C[i,j,k,l] = C44
    elif i == j and k == l:
        C[i,j,k,l] = C12

def build_stroh_submatrices(C_rot, propagation_dir, rho, v):
    """
    Construct Stroh submatrices N1, N2, N3.
    """
    n = propagation_dir / np.linalg.norm(propagation_dir)
    m = np.array([0, 0, 1])  # Surface normal
    
    Q = np.einsum('ijkm,j,k', C_rot, n, n) - rho * v**2 * np.eye(3)
    R = np.einsum('ijkm,j,k', C_rot, n, m)
    T = np.einsum('ijkm,j,k', C_rot, m, m)
    
    N1 = -np.linalg.inv(T) @ R.T
    N2 = np.linalg.inv(T)
    N3 = Q - R @ np.linalg.inv(T) @ R.T
    return N1, N2, N3

def _tensor_to_voigt(C):
    """Convert 4th-order elastic tensor to Voigt notation (6x6 matrix)."""
    # Mapping from tensor to Voigt indices
    voigt_map = {
        (0,0): 0, (1,1): 1, (2,2): 2,
        (1,2): 3, (0,2): 4, (0,1): 5
    }
    
    C_voigt = np.zeros((6,6))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    # Get Voigt indices
                    if i == j and k == l:
                        m = voigt_map[(i,i)]
                        n = voigt_map[(k,k)]
                        C_voigt[m,n] = C[i,j,k,l]
                    elif i == j:
                        m = voigt_map[(i,i)]
                        n = voigt_map[(k,l)] if k < l else voigt_map[(l,k)]
                        C_voigt[m,n] = C[i,j,k,l]
                    elif k == l:
                        m = voigt_map[(i,j)] if i < j else voigt_map[(j,i)]
                        n = voigt_map[(k,k)]
                        C_voigt[m,n] = C[i,j,k,l]
                    else:
                        m = voigt_map[(i,j)] if i < j else voigt_map[(j,i)]
                        n = voigt_map[(k,l)] if k < l else voigt_map[(l,k)]
                        C_voigt[m,n] = C[i,j,k,l]
    return C_voigt

def tensor_to_voigt(C):
    voigt_map = {(0,0):0, (1,1):1, (2,2):2, (1,2):3, (0,2):4, (0,1):5}
    C_voigt = np.zeros((6,6))
    for i, j, k, l in np.ndindex(3,3,3,3):
        m = voigt_map[(i,j)] if i == j else voigt_map[tuple(sorted((i,j)))]
        n = voigt_map[(k,l)] if k == l else voigt_map[tuple(sorted((k,l)))]
        if (i != j) or (k != l):
            C_voigt[m,n] = C[i,j,k,l] * (2 if i != j and k != l else 1)
        else:
            C_voigt[m,n] = C[i,j,k,l]
    return C_voigt

def secular_equation(v, C_rot, propagation_dir, rho):
    """Calculate the secular equation for SAW velocity."""
    # Convert elastic tensor to Voigt notation
    C_voigt = tensor_to_voigt(C_rot)
    
    # Set up the eigenvalue problem
    M = np.zeros((6, 6), dtype=complex)
    M[:3, :3] = C_voigt[0:3, 0:3]
    M[:3, 3:] = C_voigt[0:3, 3:6]
    M[3:, :3] = C_voigt[3:6, 0:3]
    M[3:, 3:] = C_voigt[3:6, 3:6]
    
    # Solve eigenvalue problem
    eigvals, eigenvectors = np.linalg.eig(M)
    
    # Extract eigenvectors corresponding to decaying solutions (Im(alpha) > 0)
    valid_indices = np.where(np.imag(eigvals) > 1e-6)[0]
    if len(valid_indices) < 3:
        return np.array([1e6])  # Return large value if not enough valid solutions
        
    # Select first 3 decaying modes
    B = eigenvectors[:, valid_indices[:3]]
    
    # Boundary condition: Traction-free surface (L = 0)
    L_matrix = B[3:, :]
    
    # Return determinant as scalar
    return np.abs(np.linalg.det(L_matrix))

def compute_saw_velocity(C_rot, propagation_dir, rho, v_st):
    """Compute SAW velocity using root finding."""
    # Initial guess near expected SAW velocity
    v_guess = 0.8 * v_st
    
    result = root(
        lambda v: secular_equation(v, C_rot, propagation_dir, rho),
        v_guess,
        method='lm',
        options={'maxiter': 1000}
    )
    
    if not result.success:
        raise ValueError("Failed to find SAW velocity")
        
    return result.x[0]

# Example usage
euler_angles = (0, 0, 0)  # No rotation
C_rot = rotate_elastic_tensor(C, euler_angles)
propagation_dir = np.array([1, 0, 0])  # Along x-axis

# Compute slowest bulk transverse velocity (v_ST)
Gamma = np.einsum('ijkm,j,k', C_rot, propagation_dir, propagation_dir)
eigvals = np.linalg.eigvalsh(Gamma)
v_st = np.sqrt(np.min(eigvals) / rho)

# Compute SAW velocity
v_saw = compute_saw_velocity(C_rot, propagation_dir, rho, v_st)
print(f"SAW Velocity: {v_saw:.2f} m/s")

SAW Velocity: 2988.52 m/s
